## Clasificarea CIFAR-10 cu Augmentarea Datelor

În acest exercițiu, ne întoarcem la CIFAR-10 și la rețelele pe care le-am construit anterior. Vom folosi augmentarea datelor în timp real pentru a încerca să îmbunătățim rezultatele noastre.

Odată ce ați terminat de parcurs caietul, experimentați cu diferite parametri de augmentare a datelor și vedeți dacă aceștia ajută (sau dimpotrivă!) performanța clasificatorului dumneavoastră.

In [1]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Datele, amestecate și împărțite între seturile de antrenare și testare:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [4]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

În Exercițiul 6, am construit două modele. Unul era mai mic (cu 181K parametri), în timp ce al doilea era mai mare (cu 1.25M parametri). Mai jos folosim modelul mai mic și îl antrenăm cu augmentare de date.

In [5]:
# Să construim un CNN folosind capabilitățile secvențiale ale lui Keras

model_1 = Sequential()


## Convoluție 5x5 cu pas de 2x2 și 32 de filtre
model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=x_train.shape[1:]))
model_1.add(Activation('relu'))

## O altă convoluție 5x5 cu pas de 2x2 și 32 de filtre
model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

## Max pooling 2x2 reduce la 3 x 3 x 32
model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

## Flatten transformă 3x3x32 în 288x1
model_1.add(Flatten())
model_1.add(Dense(512))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 16, 16, 32)        2432      
                                                                 
 activation (Activation)     (None, 16, 16, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 6, 6, 32)          25632     
                                                                 
 activation_1 (Activation)   (None, 6, 6, 32)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 32)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 3, 3, 32)          0         
                                                        

Avem încă 181K parametri, chiar dacă acesta este un model "mic".

In [8]:
batch_size = 32

# inițializăm optimizatorul RMSprop
opt = keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)

# Hai să antrenăm modelul folosind RMSprop
model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

Aici definim `ImageDataGenerator` pe care îl vom folosi pentru a furniza imagini modelului nostru în timpul procesului de antrenare. În prezent, este configurat să facă câteva deplasări și să inverseze orizontal imaginile.

In [10]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # setăm media intrării la 0 peste setul de date
    samplewise_center=True,    # setează media fiecărui eșantion la 0
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(x_train)

model_1.fit(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      steps_per_epoch=x_train.shape[0] // batch_size,
                      epochs=15,
                      validation_data=(x_test, y_test))


Epoch 1/15
1562/1562 [==============================] - 54s 34ms/step - loss: 1.6752 - accuracy: 0.3881 - val_loss: 2.1454 - val_accuracy: 0.2472
Epoch 2/15
1562/1562 [==============================] - 58s 37ms/step - loss: 1.4498 - accuracy: 0.4754 - val_loss: 2.3423 - val_accuracy: 0.2341
Epoch 3/15
1562/1562 [==============================] - 58s 37ms/step - loss: 1.3414 - accuracy: 0.5197 - val_loss: 2.0809 - val_accuracy: 0.3061
Epoch 4/15
1562/1562 [==============================] - 45s 29ms/step - loss: 1.2815 - accuracy: 0.5473 - val_loss: 2.0467 - val_accuracy: 0.3349
Epoch 5/15
1562/1562 [==============================] - 43s 28ms/step - loss: 1.2412 - accuracy: 0.5618 - val_loss: 1.7553 - val_accuracy: 0.3719
Epoch 6/15
1562/1562 [==============================] - 45s 29ms/step - loss: 1.2214 - accuracy: 0.5694 - val_loss: 2.0131 - val_accuracy: 0.3360
Epoch 7/15
1562/1562 [==============================] - 51s 33ms/step - loss: 1.1967 - accuracy: 0.5796 - val_loss: 1.8426 -

Cum se compară performanța cu antrenarea fără augmentare?

In [ ]:
Performanța modelului antrenat cu augmentare de date poate varia în funcție de setul de date și de arhitectura modelului.
În general, augmentarea datelor poate ajuta la îmbunătățirea generalizării și la reducerea suprapunerii în antrenare, ceea ce poate duce la rezultate mai bune pe setul de testare.
Cu toate acestea, efectul exact poate fi diferit în funcție de scenariu.
Pentru a evalua exact diferența de performanță, este necesar să comparați metricile de evaluare relevante (cum ar fi acuratețea sau pierderea) între modelul antrenat cu augmentare de date și modelul antrenat fără augmentare, folosind același set de date de testare.

## Exercițiu
### Rândul tău

1. Experimentează mai sus cu diferite setări ale parametrilor de augmentare a datelor. Poți să faci modelul să se comporte mai bine? Poți să-l faci să se comporte mai rău?

2. Așa cum ai făcut în Exercițiul 6, Construiește un model mai complicat cu următorul pattern:
   - Conv -> Conv -> MaxPool -> Conv -> Conv -> MaxPool -> (Flatten) -> Dense -> Clasificare Finală
   - Folosește pași de 1 pentru toate straturile de convoluție.

3. Folosește augmentarea datelor pentru a antrena acest model. Poți obține o performanță mai bună?

In [11]:
# Hai să construim o rețea neurală convoluțională folosind capabilitățile secvențiale ale Keras

# Scrie codul tău aici

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [12]:
## Verifica numarul de parametri (afiseaza summary-ul)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 10, 10, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                  

In [13]:
# Inițializăm optimizatorul RMSprop

# Hai să antrenăm modelul folosind RMSprop

from keras.datasets import cifar10
from keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [14]:
# Calculăm cantitățile necesare pentru normalizarea pe bază de caracteristici

# Antrenăm modelul pe loturile generate de datagen.flow().

from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to the range [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=True,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=0,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
)

datagen.fit(x_train)

# Train the model
history = model.fit(datagen.flow(x_train, y_train, batch_size=32),
                              steps_per_epoch=x_train.shape[0] // 32,
                              epochs=15,
                              validation_data=(x_test, y_test))


Epoch 1/15
1562/1562 [==============================] - 200s 127ms/step - loss: 1.4685 - accuracy: 0.4704 - val_loss: 1.7443 - val_accuracy: 0.3846
Epoch 2/15
1562/1562 [==============================] - 191s 122ms/step - loss: 1.0748 - accuracy: 0.6227 - val_loss: 1.1999 - val_accuracy: 0.5726
Epoch 3/15
1562/1562 [==============================] - 192s 123ms/step - loss: 0.9320 - accuracy: 0.6773 - val_loss: 1.0415 - val_accuracy: 0.6428
Epoch 4/15
1562/1562 [==============================] - 193s 124ms/step - loss: 0.8828 - accuracy: 0.6988 - val_loss: 1.3499 - val_accuracy: 0.5557
Epoch 5/15
1562/1562 [==============================] - 191s 122ms/step - loss: 0.8621 - accuracy: 0.7095 - val_loss: 2.5690 - val_accuracy: 0.3966
Epoch 6/15
1562/1562 [==============================] - 191s 122ms/step - loss: 0.8620 - accuracy: 0.7094 - val_loss: 2.9927 - val_accuracy: 0.4710
Epoch 7/15
1562/1562 [==============================] - 188s 120ms/step - loss: 0.8692 - accuracy: 0.7087 - val_